# 如何添加记忆模块

## ConversationBufferMemory 是一种最简单的memory，对聊天内容不做任何压缩和处理

In [21]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.load_memory_variables({})




{'history': ''}

In [14]:
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")
memory.load_memory_variables({})

{'history': "Human: hi!\nAI: what's up?"}

## 在LLMChain里添加记忆模块

In [1]:
import os

from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


llm = OpenAI(
    temperature=0,
    openai_api_key = os.getenv("OPENAI_API_KEY"),
    base_url = os.getenv("OPENAI_BASE_URL")
)

# 注意，在模板里，要给聊天记录留下位置，聊天记录是通过注入给模板来实现的

template = """You are a nice chatbot having a conversation with a human.

Previous conversation:
{chat_history}

New human question: {question}
Response:"""


prompt = PromptTemplate.from_template(template)

# `memory_key`要和模板里的参数保持一致

memory = ConversationBufferMemory(memory_key="chat_history")

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True, 
    memory=memory
)

conversation.invoke({"question":"hi"})
conversation.invoke({"question":"你最喜欢什么动物?"})
conversation.invoke({"question":"那么它最爱吃什么?"})

memory.load_memory_variables({})



> Entering new LLMChain chain...
Prompt after formatting:
You are a nice chatbot having a conversation with a human.

Previous conversation:


New human question: hi
Response:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
You are a nice chatbot having a conversation with a human.

Previous conversation:
Human: hi
AI:  Hello! How are you doing today?

New human question: 你最喜欢什么动物?
Response:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
You are a nice chatbot having a conversation with a human.

Previous conversation:
Human: hi
AI:  Hello! How are you doing today?
Human: 你最喜欢什么动物?
AI:  我最喜欢猫咪！它们又可爱又聪明，总是能给人带来快乐。你呢？你最喜欢什么动物？

New human question: 那么它最爱吃什么?
Response:

> Finished chain.


{'chat_history': 'Human: hi\nAI:  Hello! How are you doing today?\nHuman: 你最喜欢什么动物?\nAI:  我最喜欢猫咪！它们又可爱又聪明，总是能给人带来快乐。你呢？你最喜欢什么动物？\nHuman: 那么它最爱吃什么?\nAI:  猫咪最爱吃鱼肉和鸡肉，但它们也喜欢吃一些猫粮和零食。它们的食量通常不会很大，但它们会经常吃小量的食物。你有宠物吗？它们喜欢吃什么？'}

## 在LCEL 模式下的chain里添加 memory

In [24]:
import langchain
import os

from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.memory import ConversationBufferMemory



llm = ChatOpenAI(
    temperature=0,
    openai_api_key = os.getenv("OPENAI_API_KEY"),
    base_url = os.getenv("OPENAI_BASE_URL")
)


## 注意 return_messages = True是因为我们选择的是聊天模型包装器ChatOpenAI，memory也要对应选择返回的是messages消息体而不是str

In [25]:
memory = ConversationBufferMemory(return_messages=True)

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个中国历史专家."),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{input}")
])

memory.load_memory_variables({})

{'history': []}

In [28]:
memory = ConversationBufferMemory(return_messages=True)

chain = (
    RunnablePassthrough.assign(
        history=  RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | llm
)


input = {"input":"中国改革开放的总设计师是谁?"}
response = chain.invoke(input)
print(response)

# memory.save_context(input, {"output": response.content})
memory.load_memory_variables({})

content='中国改革开放的总设计师是邓小平。邓小平是中国共产党的杰出领导人，他在1978年提出了改革开放的政策，推动了中国经济的快速发展和社会的变革。他的改革开放政策包括引进外资、开放市场、发展私营经济等一系列措施，为中国的现代化建设奠定了基础。邓小平的改革开放政策被视为中国历史上的重大转折点，对中国的发展产生了深远的影响。'


{'history': []}

In [28]:
# memory.save_context(input, {"output": response.content})

memory.load_memory_variables({})

{'history': []}

In [29]:
input = {"input":"他是什么时候去世的?"}
response = chain.invoke(input)
print(response)

memory.load_memory_variables({})

content='很抱歉，我无法回答你的问题，因为你没有提供具体的人物信息。中国历史悠久，有许多重要人物，每个人物的去世时间都不同。如果你能提供更多的背景信息，我将尽力帮助你回答。'


{'history': []}

In [30]:
memory.save_context(input, {"output": response.content})

memory.load_memory_variables({})

{'history': [HumanMessage(content='他是什么时候去世的?'),
  AIMessage(content='很抱歉，我无法回答你的问题，因为你没有提供具体的人物信息。如果你能提供人物的姓名和背景，我将尽力为你提供相关的信息。')]}